In [6]:
using BenchmarkTools
using DRR

In [9]:
import PaddedViews

In [24]:
a = collect(reshape(1:9, 3, 3))
b = PaddedViews.PaddedView(-1, a, (1:5,1:5), (2:4,2:4))

5×5 PaddedView(-1, OffsetArray(::Matrix{Int64}, 2:4, 2:4), (1:5, 1:5)) with eltype Int64 with indices 1:5×1:5:
 -1  -1  -1  -1  -1
 -1   1   4   7  -1
 -1   2   5   8  -1
 -1   3   6   9  -1
 -1  -1  -1  -1  -1

In [23]:
a3D = collect(reshape(1:8, 2, 2,2))
d1, d2, d3 = size(a3D)
b3D = PaddedViews.PaddedView(minimum(a3D[:,1,1]), a3D, (1:(d1+2),1:(d2+2), 1:(d3+2)), (2:(d1+1),2:(d2+1),2:(d3+1)))

4×4×4 PaddedView(1, OffsetArray(::Array{Int64, 3}, 2:3, 2:3, 2:3), (1:4, 1:4, 1:4)) with eltype Int64 with indices 1:4×1:4×1:4:
[:, :, 1] =
 1  1  1  1
 1  1  1  1
 1  1  1  1
 1  1  1  1

[:, :, 2] =
 1  1  1  1
 1  1  3  1
 1  2  4  1
 1  1  1  1

[:, :, 3] =
 1  1  1  1
 1  5  7  1
 1  6  8  1
 1  1  1  1

[:, :, 4] =
 1  1  1  1
 1  1  1  1
 1  1  1  1
 1  1  1  1

In [22]:
minimum(a3D[:,1,1])

1

In [15]:
b[3,3]

5

We will need to benchmark four experiments: 

|     | Serial     | Parallel     |
|-----|------------|--------------|
| CPU | CPU Serial | CPU Parallel |
| GPU | GPU Serial | GPU Parallel |

In [5]:
# Generate voxel grid

# Load the CT volume and convert it to a regular grid with correct spacing
ct_scan, ΔX, ΔY, ΔZ = read_dicom("../data/cxr/")
voxel_grid = volume2grid(ct_scan, ΔX, ΔY, ΔZ)

# Initialize the X-ray imaging system's geometry
camera = Camera(center=(-2,-2,-2))
detector = Detector(center=(2,2,2), normal=(-2,-2,-2), height=200, width=200)

LoadError: UndefVarError: read_dicom not defined

In [ ]:
# Generate the DRR
sampling_interval = 0.01
drr = make_drr(voxel_grid, camera, detector, sampling_rate)